In [8]:
!pip install bacdive
import bacdive
import json
import pandas as pd
from google.colab import files

client = bacdive.BacdiveClient('jed.lin1015@gmail.com', '345054944Qaz!')
# holy insecure i gotta change this to a password i dont use
# alr done

-- Authentication successful --


In [12]:
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_excel(file_name)

df = df[['ID_strains']]
df.dropna(inplace=True)

def extract_shape(record):
    return record["Morphology"]["cell morphology"]['cell shape']

def normalize_shape(shape_raw):
    if not shape_raw:
        return None
    s = shape_raw.lower().strip()
    if any(word in s for word in ["rod", "bacillus"]):
        return "rod"
    if any(word in s for word in ["coccus", "spherical", "sphere"]):
        return "coccus"
    if any(word in s for word in ["spiral", "spirillum", "curved"]):
        return "spiral"
    if any(word in s for word in ["vibrio", "comma"]):
        return "vibrio"
    if "filament" in s:
        return "filamentous"
    return "other"

Saving pfam_binary_matrix.xlsx to pfam_binary_matrix (2).xlsx


In [22]:
metadata = []

for index, row in df.iterrows():
    strain_id = row['ID_strains']
    try:
        client.search(id=str(strain_id))
        for strain in client.retrieve():
            data = strain["Morphology"]["cell morphology"]["cell shape"]
            shape_std = normalize_shape(data)
            metadata.append({"ID_strains": strain_id, "shape_std": shape_std})
    except Exception as e:
        metadata.append({"ID_strains": strain_id, "shape_std": None})

output_df = pd.DataFrame(metadata)
output_file = "bacdive_morphology.xlsx"
output_df.to_excel(output_file, index=False)
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>